In [2]:
import matsim
import os
import contextily as ctx
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from my_matsim_utils import *
%matplotlib inline

In [3]:
import os
import platform

def get_data_folder_path():
    # Get the current operating system
    os_type = platform.system()
    user_name = os.getlogin()

    # Define data folder paths for different systems
    if os_type == 'Windows' and user_name == 'muaa':
        data_folder_path = f"C://Users//{user_name}//Documents//3_MIEI//2023_ABMT_Data//Zurich//"
    elif os_type == 'Linux' and user_name == 'cmuratori':
        data_folder_path = '/cluster/home/cmuratori/.../'
    elif os_type == 'Darwin':
        data_folder_path = '/Users/Marco/Library/CloudStorage/OneDrive-Persönlich/ETHZ/Agent Based Modeling/data/'
    else:
        raise Exception("Unsupported system configuration")

    return data_folder_path

### <font color ="gold"> Choose Population % to be read

In [4]:
pct = 1

In [5]:
data_folder_path = get_data_folder_path()
scenario_path = f"{data_folder_path}{pct}pct//"

# DRT_outputFolderPath = "C:\\Users\\muaa\\Documents\\3_MIEI\\2023_ABMT_Data\\Zurich\\1pct\\outputWithDRT" 
# output_folder_path = data_folder_path + f'output//output_zurich_{pct}pct//'
# output_folder_path = "C:\\Users\\muaa\\Documents\\3_MIEI\\2023_ABMT_Data\\Zurich\\simulation_output_baseline_25pct\\"
output_folder_path = "C:\\Users\\muaa\\Documents\\3_MIEI\\2023_ABMT_Data\\Zurich\\1pct\\outputWithDRT"
# output_folder_path = "C:\\Users\\muaa\\Documents\\3_MIEI\\2023_ABMT_Data\\Zurich\\simulation_output_drt_25pct\\"




### <font color ="gold"> Network

How to read in and visualize the network

In [6]:
# This reads in the network in a structure that contains two tables, one for links and one for nodes.
network = matsim.read_network(os.path.join(scenario_path, "zurich_network.xml.gz"))

# this creates a geographic dataframe
network_geo = network.as_geo('epsg:2056')

c:\Users\muaa\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
network_geo.to_file(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/network_geo.geojson', driver='GeoJSON')

### Reading Data

In [8]:
# The plan reader
plans_synt = matsim.plan_reader_dataframe(os.path.join(scenario_path, f"zurich_population_{pct}pct.xml.gz"))
# households_synt = matsim.household_reader(os.path.join(scenario_path, "zurich_households.xml.gz")) ---with some difference in the env it should work
households_synt = my_houshold_reader(os.path.join(scenario_path, "zurich_households.xml.gz"))

households_sim = my_houshold_reader(os.path.join(output_folder_path, "output_households.xml.gz"))
plans_sim = matsim.plan_reader_dataframe(os.path.join(output_folder_path, "output_plans.xml.gz"))     

In [9]:
df_households_synt = households_synt.households

df_activity_synt = plans_synt.activities
# df_plans_synt = plans.plans
df_legs_synt = plans_synt.legs
df_persons_synt = plans_synt.persons
df_routes_synt = plans_synt.routes

df_activity_sim = plans_sim.activities
# df_plans_sim = plans.plans
df_legs_sim = plans_sim.legs
df_persons_sim = plans_sim.persons
df_routes_sim = plans_sim.routes

In [10]:
# Save _synth dataframes
df_households_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_households_synt.csv', index=False)
df_activity_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_activity_synt.csv', index=False)
# df_plans_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_plans_synt.csv', index=False)
df_legs_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_legs_synt.csv', index=False)
df_persons_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_persons_synt.csv', index=False)
df_routes_synt.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_routes_synt.csv', index=False)

# Save _sim dataframes
df_activity_sim.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_activity_sim.csv', index=False)
df_legs_sim.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_legs_sim.csv', index=False)
df_persons_sim.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_persons_sim.csv', index=False)
df_routes_sim.to_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/zurich_{pct}pct/df_routes_sim.csv', index=False)